## ToyTree objects

The main class object users interact with in toytree is called a `ToyTree`. This object contains a number of useful functions for interacting with the underlying `TreeNode` structure (e.g., rooting, dropping tips) and for drawing trees and adding data from the tree (e.g., support values) to the plots. The link between tree structure and the data used to build tree drawings is tightly linked in toytree with the goal of making it very difficult for users to accidentally plot tip or node labels in an incorrect order. This section of the tutorial is primarily about how ToyTree objects store data, and how to access it easily using their functions. 

In [1]:
import toytree
import toyplot
import numpy as np

In [2]:
# load a tree for this tutorial
tre = toytree.tree("https://eaton-lab.org/data/Cyathophora.tre")

### Manipulating ToyTrees

ToyTrees provide a number of fnctions for modifying the tree structure. All of these methods return a modified copy of the object -- they do not change it in place. This means your original tree is not effected unless you store the new tree to the same variable the old tree was stored in. For example, to root the tree that we loaded in above I would store the new rooted tree under a new variable (e.g., rtre) to still retain a copy of old unrooted tree. 

In [3]:
# root a tre (more detail on this below...)
rtre = tre.root(["32082_przewalskii", "33588_przewalskii"])

#### Selecting parts of the tree to manipulate

As you can see above, and in examples below, we use tip names to designate the location in the tree where it should be manipulated. In this case, by selecting multiple tips we are indicating that the tree modification (rooting) should occur on the edge leading to their most recent common ancestor. 

But why does toytree use tip names instead of node names to indicate the tree manipulation point? Well, using node indices (e.g., idx labels) would be a reasonable alternative, but it turns out this would likely be more error prone for users. This is because as the tree is modified (e.g., if tips are dropped) the node indices will change. In contrast, the relationships among tips (i.e., who shares a more recent common ancestor with whom) does not change with any of these tree modifications. Thus, you can reliably chain together multiple tree modification functions when using tip labels (i.e., clade identities) without having to check any attributes of the intermediate trees that may have changed. For an example where this becomes clear, see [Chaining many functions and arguments](#Chaining-functions-and-arguments).

#### Fuzzy name matching

Toytrees functions allow for a variety of methods to simplify the task of listing multiple tip names to represent a clade. To create the name list without having to type each name out by hand, which becomes tedious for large trees, you can use fuzzy name matching. The three options are to write each name into a list using the `names` argument; to select samples based on a shared unique string sequence in their names with `wildcard`; or using a `regex` (regular expression) statement to match samples using more complex name patterns. When selecting multiple samples the root will be placed on the edge leading to their common ancestor, and so an error will be raised if the samples do not form a monophyletic clade. 



In [4]:
rtre = tre.root(names=["32082_przewalskii", "33588_przewalskii"])
rtre = tre.root(wildcard="prz")
rtre = tre.root(regex="[0-9]*_przewalskii")

If you really want to select parts of the tree using nodes because maybe the tip names are very hard to match then this can be done usign the `get_tip_labels()` function to build a list of tip names from a node idx label. If you enter an `idx` argument to this function it will return a list of names descended from the node. As stated above, this method is less preferred than to match tip names, since node idx labels and names will change when the tree structure changes. 

In [5]:
# view unrooted tree with node labels
tre.draw(node_labels='idx');

38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 33588_przewalskii 32082_przewalskii 30686_cyathophylla 29154_superba 41478_cyathophylloides 41954_cyathophylloides idx: 13
name: i13
dist: 0.0062
support: 100
height: 0.0243 13 idx: 14
name: i14
dist: 0.0007
support: 96
height: 0.0305 14 idx: 15
name: i15
dist: 0.0022
support: 100
height: 0.0290 15 idx: 16
name: i16
dist: 0.0008
support: 99
height: 0.0313 16 idx: 17
name: i17
dist: 0.0010
support: 100
height: 0.0320 17 idx: 18
name: i18
dist: 0.0054
support: 100
height: 0.0331 18 idx: 19
name: i19
dist: 0.0359
support: 100
height: 0.0026 19 idx: 20
name: i20
dist: 0.0030
support: 100
height: 0.0385 20 idx: 21
name: i21
dist: 0.0024
support: 100
height: 0.0391 21 idx: 22
name: i22
dist: 0.0094
support: 100
height: 0.0414 22 idx: 23
name: i23
dist: 0.0000
support: 100
height: 0.0509 23

In [6]:
# get list of tips descended from some node label
tipnames = tre.get_tip_labels(idx=19)
tipnames

['33588_przewalskii', '32082_przewalskii']

In [7]:
# now root on this list of names (not the node idx labels changed)
tre.root(names=tipnames).draw(node_labels='idx');

38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 41954_cyathophylloides 41478_cyathophylloides 30686_cyathophylla 29154_superba 33588_przewalskii 32082_przewalskii idx: 13
name: i13
dist: 0.0062
support: 100
height: 0.0011 13 idx: 14
name: i14
dist: 0.0007
support: 96
height: 0.0073 14 idx: 15
name: i15
dist: 0.0022
support: 100
height: 0.0058 15 idx: 16
name: i16
dist: 0.0008
support: 99
height: 0.0080 16 idx: 17
name: i17
dist: 0.0010
support: 100
height: 0.0088 17 idx: 18
name: i17
dist: 0.0094
support: 100
height: 0.0028 18 idx: 19
name: i21
dist: 0.0024
support: 100
height: 0.0099 19 idx: 20
name: i18
dist: 0.0054
support: 100
height: 0.0098 20 idx: 21
name: i22
dist: 0.0030
support: 100
height: 0.0122 21 idx: 22
name: i20
dist: 0.0179
support: 100
height: 0.0152 22 idx: 23
name: i19
dist: 0.0179
support: 100
height: 0.0152 23 idx: 24
name: i23
dist: 0.0000
support: 100
height: 0.0331 24

#### Rooting 
You can root toytrees using the `.root()` function call. This takes as an argument either a single tip name, or a list of tip names. You can use the fuzzy name matching options to match multiple tip names, as shown below. 

In [8]:
rtre0 = tre.root(names=["32082_przewalskii", "33588_przewalskii"])
rtre1 = tre.root(wildcard="prz")
rtre2 = tre.root(regex="[0-9]*_przewalskii")
rtre2.draw(node_labels='idx');

38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 41954_cyathophylloides 41478_cyathophylloides 30686_cyathophylla 29154_superba 33588_przewalskii 32082_przewalskii idx: 13
name: i13
dist: 0.0062
support: 100
height: 0.0011 13 idx: 14
name: i14
dist: 0.0007
support: 96
height: 0.0073 14 idx: 15
name: i15
dist: 0.0022
support: 100
height: 0.0058 15 idx: 16
name: i16
dist: 0.0008
support: 99
height: 0.0080 16 idx: 17
name: i17
dist: 0.0010
support: 100
height: 0.0088 17 idx: 18
name: i17
dist: 0.0094
support: 100
height: 0.0028 18 idx: 19
name: i21
dist: 0.0024
support: 100
height: 0.0099 19 idx: 20
name: i18
dist: 0.0054
support: 100
height: 0.0098 20 idx: 21
name: i22
dist: 0.0030
support: 100
height: 0.0122 21 idx: 22
name: i20
dist: 0.0179
support: 100
height: 0.0152 22 idx: 23
name: i19
dist: 0.0179
support: 100
height: 0.0152 23 idx: 24
name: i23
dist: 0.0000
support: 100
height: 0.0331 24

There is also a function `.unroot()` to remove the root node from trees. This creates a polytomy at the root. Technically there still exists a point on the treenode structure that we refer to as the root, but it does not appear in drawings.

In [9]:
# an unrooted tree
rtre0.unroot().draw();

38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 41954_cyathophylloides 41478_cyathophylloides 30686_cyathophylla 29154_superba 33588_przewalskii 32082_przewalskii

#### Drop tips
Dropping tips from a tree retains the structure of the remaining nodes in the tree. Here again you can use fuzzy name matching to select the tips you wish to drop from the tree. In this case the names that are selected with matching do not have to form a monophyletic clade, however, if you select to remove all tips in the tree then it will raise an error. 

In [10]:
rtre0.drop_tips(wildcard="cyatho").draw();

29154_superba 38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 33588_przewalskii 32082_przewalskii

#### Ladderize
By default toytrees are ladderized unless you change the tip order in some way, by either entering a fixed_order for tip labels, by dropping tips from the tree, or by rotating nodes. If you want to return a tree to being ladderized you can do so with the `.ladderize()` function. 

In [11]:
# dropping tips unladderized the tree, so we re-ladderized it before plotting
rtre0.drop_tips(wildcard="cyatho").ladderize().draw();


38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 29154_superba 33588_przewalskii 32082_przewalskii

#### Rotate nodes 
Rotating nodes of the tree does not affect the actual tree structure (e.g., the newick structure does not change), it simply affects the order of tips when the tree is drawn. You can rotate nodes by entering tip names as in the previous examples using either names, wildcard, or regex. The names must form a monophyletic clade for one of the descendants of the node you wish to rotate. Rotating nodes for plotting is usually done for some aesthetic reason, such as aligning tips better with geography or trait values plotted on the tips of the tree. 

In [12]:
rtre0.rotate_node(wildcard="prz").draw();

33588_przewalskii 32082_przewalskii 38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 41954_cyathophylloides 41478_cyathophylloides 30686_cyathophylla 29154_superba

#### Resolve polytomy
This method should generally not be used much unless needed. The problem is that you usually don't know what to set the branch length to for the new edge when you split a polytomy. If the tree is unrooted then you should use `.root()` instead to root it. If you have a hard polytomy in the tree and need to resolve it then this will resolve all polytomies in the tree. You can change whether what the defealt .dist and .support values will be on the new node. 

In [13]:
toytree.tree("((a,b,c),d);").resolve_polytomy(dist=1.).draw();

a c b d

### Chaining functions and arguments
Because the tree modification calls in toytrees always return a copy of the object, you can chain together many of these functions when building a plot. This is especially nice if you are only modifying the tree temporarily for the purpose of plotting (e.g., rotating nodes), and so you don't need to store the intermediate trees. It's kind of analagous to using pipes in bash programming. 

When chaining many function calls and plotting styles together in toytree code it is best to use good coding practices. In the example below I split each function call and style option over a separate line. This makes the code more readable, and easier to debug, since you can comment out a line at a time to examine its effect without it breaking the rest of the command. The parentheses surrounding the main function calls makes this possible. 

In [14]:
# readable style for writing long draw functions
canvas, axes = (
    tre
    .root(wildcard="prz")
    .drop_tips(wildcard="superba")
    .rotate_node(wildcard="30686")
    .draw(
        tip_labels_align=True,
        edge_style={
            "stroke": toytree.colors[3],
        }
    )
)

38362_rex 39618_rex 35236_rex 35855_rex 40578_rex 30556_thamno 33413_thamno 41954_cyathophylloides 41478_cyathophylloides 30686_cyathophylla 33588_przewalskii 32082_przewalskii

### Attributes and functions

In [15]:
rtre.get_tip_labels()           # list of labels in node-plot order
rtre.get_tip_coordinates()      # array of tip plot coordinates in idx order
rtre.get_node_values()          # list in node-plot order
rtre.get_node_dict()            # dict mapping idx:name for each tip
rtre.get_node_coordinates()     # array of node plot coordinates in idx order
rtre.get_edge_lengths();        # list of node.dist values in node plot order

In [16]:
rtre.is_bifurcating()           # boolean
rtre.is_rooted();               # boolean

In [17]:
rtre.nnodes                     # number of nodes in the tree
rtre.ntips                      # number of tips in the tree
rtre.newick                     # the newick representation of the tree
rtre.features                   # list of node features that can be accessed 
rtre.style;                     # dict of plotting style of tree 

### Saving/writing ToyTrees


In [18]:
# if no file handle is entered then the newick string is returned
rtre.write()

'((32082_przewalskii:0.00259326,33588_przewalskii:0.00247134)100:0.0179371,(((29154_superba:0.00634237,30686_cyathophylla:0.00669945)100:0.00237995,(41478_cyathophylloides:5.28218e-05,41954_cyathophylloides:8.88803e-05)100:0.00941021)100:0.00297626,(33413_thamno:0.00565358,(30556_thamno:0.00653218,((40578_rex:0.00335406,35855_rex:0.00339963)100:0.00223,(35236_rex:0.00580525,(39618_rex:0.000962081,38362_rex:0.00109218)100:0.00617527)96:0.0007389)99:0.000783365)100:0.0010338)100:0.00538723)100:0.0179371);'

In [19]:
# the fmt (format) options write different newick formats.
rtre.write(fmt=9)

'((32082_przewalskii,33588_przewalskii),(((29154_superba,30686_cyathophylla),(41478_cyathophylloides,41954_cyathophylloides)),(33413_thamno,(30556_thamno,((40578_rex,35855_rex),(35236_rex,(39618_rex,38362_rex)))))));'

In [20]:
# write to file 
rtre.write("mytree.tre", fmt=0)